In [2]:
import numpy as np

# Define a function to compute a Householder transformation that reflects x into a multiple of e1
def house(x):
    # Get the length and norm of x
    n = len(x)
    normx = np.linalg.norm(x)
    
    # Compute the vector u and the scalar tau
    u = x.copy()
    u[0] = u[0] + np.sign(x[0]) * normx
    tau = 2 * u[0]**2 / (u[0]**2 + np.sum(u[1:]**2))
    
    # Normalize u by its first element
    u = u / u[0]
    
    # Return u and tau
    return u, tau

# Define a function to apply a Householder transformation to a matrix from the left or right
def apply_house(H, A, side):
    # Get the dimensions of A
    m, n = A.shape
    
    # Get the vector u and the scalar tau from H
    u, tau = H
    
    # Apply the transformation from the left
    if side == "left":
        # Compute the matrix product u * (A^H * u)
        temp = u.reshape(-1,1) @ (A.T @ u).reshape(1,-1)
        
        # Update A as A - (1/tau) * u * temp
        A = A - (1/tau) * u.reshape(-1,1) @ temp
        
    # Apply the transformation from the right
    elif side == "right":
        # Compute the matrix product (A * u) * u^H
        temp = (A @ u).reshape(-1,1) @ u.reshape(1,-1)
        
        # Update A as A - (1/tau) * temp * u^H
        A = A - (1/tau) * temp @ u.reshape(-1,1).T
        
    # Return the updated matrix
    return A

# Define a function to perform the Householder reduction to bidiagonal form
def house_bidiag(A):
    # Get the dimensions of A
    m, n = A.shape
    
    # Initialize the matrices U and V to identity matrices
    U = np.eye(m)
    V = np.eye(n)
    
    # Initialize the matrix B to a copy of A
    B = A.copy()
    
    # Loop over the columns of B
    for k in range(n-1):
        # Compute the Householder transformation H_k for the subvector B(k:m,k)
        H_k = house(B[k:,k])
        
        # Apply H_k to the submatrix B(k:m,k:n) from the left
        B[k:,k:] = apply_house(H_k, B[k:,k:], "left")
        
        # Apply H_k to the submatrix U(k:m,k:m) from the left
        U[k:,k:] = apply_house(H_k, U[k:,k:], "left")
        
        # Compute the Householder transformation G_k for the subvector B(k,k+1:n)^T
        G_k = house(B[k,k+1:].T)
        
        # Apply G_k to the submatrix B(k:m,k+1:n) from the right
        B[k:,k+1:] = apply_house(G_k, B[k:,k+1:], "right")
        
        # Apply G_k to the submatrix V(k+1:n,k+1:n) from the right
        V[k+1:,k+1:] = apply_house(G_k, V[k+1:,k+1:], "right")
        
    # Extract the diagonal and superdiagonal elements of B
    d = np.diag(B)
    s = np.diag(B, k=1)
    
    # Construct the bidiagonal matrix B
    B = np.diag(d) + np.diag(s, k=1)
    
    # Return B, U, and V
    return B, U, V
